In [3]:
import  pandas  as  pd
import  numpy  as  np 
import  time 
import  turicreate  as  tc 
from  sklearn.model_selection  import train_test_split

import  sys

In [4]:
customers = pd.read_csv('../data/recommend_1.csv')
transactions = pd.read_csv('../data/trx_data.csv')

In [5]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [6]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [7]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [8]:
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


In [9]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.29 minutes


In [10]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [11]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [12]:
data_dummy = create_data_dummy(data)

In [13]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
(df_matrix.shape)

(24429, 300)

In [16]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [18]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [19]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [20]:
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [21]:
train_data

customerId,productId,purchase_count
4796,33,1
1803,183,1
22313,25,1
8259,33,1
21720,140,2
27253,20,2
9277,117,1
3027,126,1
6453,19,5
28480,285,1


In [22]:
test_data

customerId,productId,purchase_count
4747,96,2
17667,148,1
16877,64,1
14933,26,2
13953,67,1
3128,152,1
1484,298,1
27168,285,1
21372,208,1
1953,229,3


In [23]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [24]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [25]:
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [26]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23302 users and 300 items.

Data prepared in: 0.193647s

106868 observations to process; with 300 unique items.

In [27]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 41881.3

recommendations finished on 2000/62483 queries. users per second: 69976.6

recommendations finished on 3000/62483 queries. users per second: 93662.2

recommendations finished on 4000/62483 queries. users per second: 113189

recommendations finished on 5000/62483 queries. users per second: 129692

...

recommendations finished on 58000/62483 queries. users per second: 126925

recommendations finished on 59000/62483 queries. users per second: 127476

recommendations finished on 60000/62483 queries. users per second: 127480

recommendations finished on 61000/62483 queries. users per second: 127827

recommendations finished on 62000/62483 queries. users per second: 126969

In [28]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [29]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [30]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23328 users and 300 items.

Data prepared in: 0.125875s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 65720.3

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     25    |  1.0  |  1   |
|    1553    |     55    |  1.0  |  2   |
|    1553    |    195    |  1.0  |  3   |
|    1553    |    274    |  1.0  |  4   |
|    1553    |     36    |  1.0  |  5   |
|    1553    |    271    |  1.0  |  6   |
|    1553    |     54    |  1.0  |  7   |
|    1553    |     2     |  1.0  |  8   |
|    1553    |    161    |  1.0  |  9   |
|    1553    |    179    |  1.0  |  10  |
|   20400    |     25    |  1.0  |  1   |
|   20400    |     55    |  1.0  |  2   |
|   20400    |    195    |  1.0  |  3   |
|   20400    |    274    |  1.0  |  4   |
|   20400    |     36    |  1.0  |  5   |
|   20400    |    271    |  1.0  |  6   |
|   20400    |     54    |  1.0  |  7   |
|   20400    |     2     |  1.0  |  8   |
|   20400    |    161    |  1.0  |  9   |
|   20400    |    179    |  1.0  |  10  |
|   19750    |     25    |  1.0  |

In [31]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23348 users and 300 items.

Data prepared in: 0.129607s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 97761.3

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7601351351351351 |  1   |
|    1553    |    247    |  0.3388429752066115 |  2   |
|    1553    |    230    |  0.331386861313868  |  3   |
|    1553    |    294    | 0.27518796992481176 |  4   |
|    1553    |    125    |  0.2562499999999996 |  5   |
|    1553    |    276    |     0.236328125     |  6   |
|    1553    |    204    | 0.22823529411764693 |  7   |
|    1553    |    213    | 0.22322775263951775 |  8   |
|    1553    |    155    |  0.2214285714285715 |  9   |
|    1553    |    248    | 0.21944444444444444 |  10  |
|   20400    |    226    |  0.7601351351351351 |  1   |
|   20400    |    247    |  0.3388429752066115 |  2   |
|   20400    |    230    |  0.331386861313868  |  3   |
|   20400    |    294    | 0.27518796992481176 |  4   |
|   20400    |    125    |  0.2562499999999996 |

In [32]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
37     3.070312
132    3.036364
34     3.022642
0      2.965258
3      2.846639
27     2.833333
248    2.829787
110    2.798817
32     2.705584
230    2.666667
245    2.663043
10     2.567647
226    2.536765
58     2.477778
54     2.443262
252    2.418182
129    2.417582
82     2.411504
87     2.410256
18     2.373737
Name: purchase_count, dtype: float64

In [33]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23302 users and 300 items.

Data prepared in: 0.13538s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.20488s

recommendations finished on 1000/1000 queries. users per second: 12727.7

In [34]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23328 users and 300 items.

Data prepared in: 0.123685s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.039193s

recommendations finished on 1000/1000 queries. users per second: 22082.9

In [35]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23348 users and 300 items.

Data prepared in: 0.165755s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.045272s

recommendations finished on 1000/1000 queries. users per second: 32600.9

In [36]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23302 users and 300 items.

Data prepared in: 0.142672s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.118728s

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |     37    | 3.0689047574996935 |  1   |
|    1553    |    132    | 3.036363636363636  |  2   |
|    1553    |     34    | 3.001532211562374  |  3   |
|    1553    |     0     | 2.959595297871624  |  4   |
|    1553    |     3     | 2.8457763401650564 |  5   |
|    1553    |     27    | 2.833333333333333  |  6   |
|    1553    |    248    | 2.8283896243318596 |  7   |
|    1553    |    110    | 2.7921359304493003 |  8   |
|    1553    |     32    | 2.705583756345177  |  9   |
|    1553    |    245    | 2.663043478260869  |  10  |
|   20400    |     37    | 3.0703124999999987 |  1   |
|   20400    |    132    | 3.0232558001171457 |  2   |
|   20400    |     34    | 3.0226415094339636 |  3   |
|   20400    |     0     | 2.963866149483715  |  4   |
|   20400    |     3     | 2.845030522146143  |  5   |
|   20400 

recommendations finished on 1000/1000 queries. users per second: 49019.6

In [37]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23328 users and 300 items.

Data prepared in: 0.135884s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.058148s

recommendations finished on 1000/1000 queries. users per second: 27587.7

In [38]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23348 users and 300 items.

Data prepared in: 0.145018s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.108401s

recommendations finished on 1000/1000 queries. users per second: 38553.5

In [39]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [40]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13915 queries. users per second: 29454.2

recommendations finished on 2000/13915 queries. users per second: 47745.2

recommendations finished on 3000/13915 queries. users per second: 57021.2

recommendations finished on 4000/13915 queries. users per second: 61885

...

recommendations finished on 10000/13915 queries. users per second: 75218.5

recommendations finished on 11000/13915 queries. users per second: 73622.4

recommendations finished on 12000/13915 queries. users per second: 75196.5

recommendations finished on 13000/13915 queries. users per second: 70771.4


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.004958677685950414 | 0.0025230605455009458 |
|   2    |  0.003557312252964426 | 0.0036663967876514173 |
|   3    |  0.003665109593963354 | 0.0054007875615678675 |
|   4    | 0.0072044556234279545 |  0.01565255709199353  |
|   5    |  0.00734459216672661  |  0.019846729191303945 |
|   6    |  0.006491795424601766 |  0.02108759636102489  |
|   7    |   0.0056567937990863  |  0.021298000800678358 |
|   8    |  0.005353934602946457 |  0.02285315710682278  |
|   9    |  0.005182257356170455 |  0.02569016199916363  |
|   10   |  0.004944304707150586 |  0.027525910791434148 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0884043576848028

Per User RMSE (best)
+------------+-----------------------+------

In [41]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13846 queries. users per second: 76173.1

recommendations finished on 2000/13846 queries. users per second: 96922.7

recommendations finished on 3000/13846 queries. users per second: 104055

recommendations finished on 4000/13846 queries. users per second: 113691

...

recommendations finished on 10000/13846 queries. users per second: 110994

recommendations finished on 11000/13846 queries. users per second: 109736

recommendations finished on 12000/13846 queries. users per second: 104031

recommendations finished on 13000/13846 queries. users per second: 101200


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.006355626173624149 | 0.0033073585778338036 |
|   2    |  0.005055611729019212 |  0.005221701236795863 |
|   3    |  0.004477827531417025 |  0.006940948734607552 |
|   4    |  0.004243102701141113 |  0.00885040604102539  |
|   5    |  0.004290047667196328 |  0.010776361686991018 |
|   6    |  0.004405604506716703 |  0.012931692536624633 |
|   7    | 0.0046222735808176115 |  0.016067031606529652 |
|   8    |  0.004622273580817562 |  0.018641662065386602 |
|   9    |  0.004638323141862116 |  0.02088771475922203  |
|   10   |  0.004607828975877538 |  0.02272757597415598  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |

In [42]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13852 queries. users per second: 72558.4

recommendations finished on 2000/13852 queries. users per second: 91524.8

recommendations finished on 3000/13852 queries. users per second: 97481.7

recommendations finished on 4000/13852 queries. users per second: 104849

...

recommendations finished on 10000/13852 queries. users per second: 55946.2

recommendations finished on 11000/13852 queries. users per second: 57128.9

recommendations finished on 12000/13852 queries. users per second: 53990.6

recommendations finished on 13000/13852 queries. users per second: 55597.8


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0023101357204735806 | 0.0014486476080469747 |
|   2    | 0.0025267109442679778 | 0.0026233390170005808 |
|   3    | 0.0024063913754933043 | 0.0038429783791464446 |
|   4    | 0.0023462315911059795 |  0.005124381786596623 |
|   5    | 0.0024978342477620535 |  0.006619954026465717 |
|   6    |  0.002574838771777847 |  0.00804801448751782  |
|   7    |  0.002537024050162944 |  0.009255507302720732 |
|   8    |  0.002644022523823268 |  0.011129169463705997 |
|   9    | 0.0025267109442679747 |  0.011953303819099377 |
|   10   |  0.002512272596015029 |   0.012987020799972   |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1337687155316119

Per User RMSE (best)
+------------+------------------------+-----

In [43]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.140153s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.054443s

recommendations finished on 1000/1000 queries. users per second: 33437

In [44]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,1,0.103482,1
1,1553,2,0.093467,2
2,1553,35,0.084576,3
3,1553,33,0.066861,4
4,1553,61,0.065126,5


In [45]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

In [48]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('../output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [50]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 39843.8

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|294|125|276|204|213|155|248
11,226|247|230|294|125|276|204|155|248|165
12,226|247|230|294|125|276|204|213|155|248
16,226|247|230|294|125|276|204|213|155|248
21,226|247|230|294|125|276|204|213|155|248


In [51]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [57]:
customer_recomendation(21)

recommendedProducts    226|247|230|294|125|276|204|213|155|248
Name: 21, dtype: object